In [1]:
# TODO 
# deploy as a t2
# try only returning a single item from the result tuple and return it as a list

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
import numpy as np
import pandas as pd

import torch
from  torch import nn

In [4]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [5]:
y = pd.read_csv(labels_path, index_col='filename')

In [6]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [7]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [8]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [9]:
# data.show_batch(3, figsize=(6,6))

In [10]:
learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=MSELossFlat()
)#.to_fp16()

In [11]:
learn.path = Path('..')

In [12]:
# learn.lr_find()
# learn.recorder.plot()

In [13]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(
#     30, 
#     slice(1e-2), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=3)
#     ]
# )

In [14]:
# learn = learn.load('fastai_model')
# learn = learn.to_fp32()
# learn.save('fastai_model')
# learn.export('models/export.pkl')

In [15]:
learn = load_learner('../models', 'export.pkl')

In [16]:
# learn.show_results(rows=10)

### Bentoml Route

In [17]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

@env(pip_dependencies=['gevent', 'pillow==6.2.2', 'fastai'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return [result[0], result[1].tolist(), result[2].tolist()]

Overwriting pet_regression.py


In [18]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2020-01-15 00:17:07,146] INFO - BentoService bundle 'PetRegression:20200115001404_6FC750' created at: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/bentoml-temp-1m62fpxq
[2020-01-15 00:17:11,419] INFO - BentoService bundle 'PetRegression:20200115001404_6FC750' created at: /Users/HenryDashwood/bentoml/repository/PetRegression/20200115001404_6FC750


In [31]:
from bentoml import load

service = load(saved_path)

# service.predict(data.get(0))

[2020-01-15 00:24:05,331] WARNING - Module `pet_regression` already loaded, using existing imported module.


In [20]:
!pip install --upgrade {saved_path}

Processing /Users/HenryDashwood/bentoml/repository/PetRegression/20200115001404_6FC750
  Created wheel for PetRegression: filename=PetRegression-20200115001404_6FC750-cp37-none-any.whl size=51630640 sha256=d36888f198a7811402a817b7db833343bf17111dbd3fd731ca864c2580334696
  Stored in directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/pip-ephem-wheel-cache-zdoybre5/wheels/a5/9a/38/d0bffe1c47434ff09f814afa90078daeab1f2ec66a743cb7a2
Successfully built PetRegression
  Found existing installation: PetRegression 20200111142212-CD8126
    Uninstalling PetRegression-20200111142212-CD8126:
      Successfully uninstalled PetRegression-20200111142212-CD8126


In [30]:
# !PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

In [22]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20200115001404_6FC750


In [28]:
!bentoml deployment create facial-features-detector --bento {bento_tag} --platform aws-sagemaker --api-name predict --region us-east-1 --instance-type ml.t2.medium

[2020-01-15 00:22:36,955] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-01-15 00:22:36,956] INFO - 

[2020-01-15 00:22:36,956] INFO -  ---> 406f2b43ea59

[2020-01-15 00:22:36,957] INFO - Step 2/11 : EXPOSE 8080
[2020-01-15 00:22:36,957] INFO - 

[2020-01-15 00:22:36,957] INFO -  ---> Using cache

[2020-01-15 00:22:36,958] INFO -  ---> c23188fe3fc7

[2020-01-15 00:22:36,958] INFO - Step 3/11 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential     && apt-get install -y nginx      && rm -rf /var/lib/apt/lists/*
[2020-01-15 00:22:36,958] INFO - 

[2020-01-15 00:22:36,959] INFO -  ---> Using cache

[2020-01-15 00:22:36,959] INFO -  ---> 9b55024636fa

[2020-01-15 00:22:36,959] INFO - Step 4/11 : RUN conda install pip numpy scipy       && pip install gunicorn gevent
[2020-01-15 00:22:36,960] INFO - 

[2020-01-15 00:22:36,960] INFO -  ---> Using cache

[2020-01-15 00:22:36,960] INFO -  ---> acbb0c3

In [29]:
!bentoml --verbose deployments delete facial-features-detector --force

[2020-01-15 00:23:46,387] DEBUG - Using BentoML with local Yatai server
[2020-01-15 00:23:46,580] DEBUG - Upgrading tables to the latest revision
[2020-01-15 00:23:48,108] ERROR - AWS ClientError - operation: DeleteEndpoint, code: ValidationException, message: Could not find endpoint "arn:aws:sagemaker:us-east-1:235858150097:endpoint/dev-facial-features-detector".
Successfully deleted deployment "facial-features-detector"


In [38]:
import json
import boto3
runtime = boto3.Session().client(service_name='sagemaker-runtime')

img = open('../data/resized_images/Abyssinian_1.jpg', 'rb').read()
    
response = runtime.invoke_endpoint(
    EndpointName='dev-facial-features-detector', 
    Body=bytearray(img),
)

result = response['Body'].read()
result = json.loads(result)

In [39]:
result['result']

'(ImagePoints (320, 320), tensor([[-0.4128,  0.1322],\n        [-0.7365,  0.1464],\n        [-0.6189,  0.2665],\n        [-0.7248,  0.4663],\n        [-0.4005,  0.3247],\n        [-0.2695,  0.2497],\n        [-0.3914,  0.2126],\n        [-0.4023,  0.3228],\n        [-0.2775,  0.2574],\n        [-0.2962,  0.2582],\n        [-0.2722,  0.2691]]), tensor([-0.4128,  0.1322, -0.7365,  0.1464, -0.6189,  0.2665, -0.7248,  0.4663,\n        -0.4005,  0.3247, -0.2695,  0.2497, -0.3914,  0.2126, -0.4023,  0.3228,\n        -0.2775,  0.2574, -0.2962,  0.2582, -0.2722,  0.2691]))'

In [40]:
str(learn.predict(data.get(0)))

'(ImagePoints (320, 320), tensor([[-0.5133, -0.5090],\n        [-0.7960, -0.4386],\n        [-0.8005, -0.2425],\n        [-0.8656, -0.0839],\n        [-0.5567, -0.0082],\n        [-0.2026, -0.2832],\n        [-0.5261, -0.3297],\n        [-0.5509, -0.1878],\n        [-0.2745, -0.3370],\n        [-0.3003, -0.2733],\n        [-0.3027, -0.2223]]), tensor([-0.5133, -0.5090, -0.7960, -0.4386, -0.8005, -0.2425, -0.8656, -0.0839,\n        -0.5567, -0.0082, -0.2026, -0.2832, -0.5261, -0.3297, -0.5509, -0.1878,\n        -0.2745, -0.3370, -0.3003, -0.2733, -0.3027, -0.2223]))'